In [ ]:
import cv2
import tensorflow.keras as keras
import tensorflow as tf
import math
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow
import random
import copy
 
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd  
 
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import AffinityPropagation, DBSCAN, KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
len_single = 5
len_pair = 1

In [ ]:
## belief vectors
BV12 = tf.Variable(np.random.randn(1,len_pair),trainable = True, dtype = tf.float64)
BV13 = tf.Variable(np.random.randn(1,len_pair ), trainable = True, dtype = tf.float64)
BV14 = tf.Variable(np.random.randn(1,len_pair), trainable = True, dtype = tf.float64)
BV23 = tf.Variable(np.random.randn(1,len_pair ), trainable = True, dtype = tf.float64)
BV24 = tf.Variable(np.random.randn(1,len_pair), trainable = True, dtype = tf.float64)
BV34 = tf.Variable(np.random.randn(1,len_pair ), trainable = True, dtype = tf.float64)

In [ ]:
marker = '1_5'

In [ ]:
 
#input
folder = '2-d_colliding'
#/content/drive/My Drive/VIPUL COLAB AND DATA/vipul_data
#path = '/content/drive/My Drive/VIPUL COLAB AND DATA/'
arr = np.load('/content/drive/My Drive/VIPUL COLAB AND DATA/collision_and_pass_data/' + folder +'/0.npy')
pos_arr = arr[3:, 0:8]
vel_arr = arr[3:, 8:16]
 
 
  
pos__1 = pos_arr[:,0:2]
pos__2 = pos_arr[:,2:4]
pos__3 = pos_arr[:,4:6]
pos__4 = pos_arr[:,6:8]
 
vel__1 = vel_arr[:,8:10]
vel__2 = vel_arr[:,10:12]
vel__3 = vel_arr[:,12:14]
vel__4 = vel_arr[:,14:16]
 
print( pos__1.shape,   pos_arr.shape,  vel__1.shape)
print(arr[3:])

In [ ]:
def distance(a, b):
  return np.sqrt(np.sum((a-b)**2))

In [ ]:
def rel_vel_mag(vel_a,vel_b):
  return np.sqrt(np.sum((vel_a-vel_b)**2))

In [ ]:
lst = [[], [], [], [], [], []]
pairs = [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)]
r = 1.5
dist_for_min = 2*r + 0.1
dist = 4.2
n = 6
for i in range(n):
  start = -1
  end = 1
  interval = [0,0]
  j = 0
  min_dist = 30
  while(j < pos__1.shape[0]):
    if(start == -1 and distance(pos_arr[j,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[j, 2*pairs[i][1]-2 : 2*pairs[i][1]]) < dist):
      print(i,j,  "start started")
      interval[0] = j
      start = 1
      end = -1
      min_dist = 30
    elif (start == 1 and end == -1):
      if(distance(pos_arr[j,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[j, 2*pairs[i][1]-2 : 2*pairs[i][1]]) > dist):
        if(min_dist < dist_for_min):
          print(i, j, "exited from start, and saved")
          #save and set start = -1, end = 1
          interval[1] = j;
          lst[i].append(interval)
          interval = [0,0]
          start = -1
          end = 1
        else:
          #dont save and set start to -1, end = 1
          print(i, j, "exited from start, not saved")
          start = -1
          end = 1
          interval = [0,0]
      else:
        min_dist = min(min_dist, distance(pos_arr[j,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[j, 2*pairs[i][1]-2 : 2*pairs[i][1]]))
    j += 1
 
 
print(lst)
 
popable = []
for i in range(len(lst)):
  for j in range(len(lst[i])):
    if(lst[i][j][0] == 0) :
      popable.append(i)
for i in popable:
  lst[i].pop(0)
print(lst)

In [ ]:
for i in range(6):
  for j in range(len(lst[i])):
    print("/////////////////////////////////////////////////",i , "///////////////////////", j )
    start = lst[i][j][0]
    end = lst[i][j][1]
    print([start, end])
    for k in range(start, end):
      print(distance(pos_arr[k,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[k, 2*pairs[i][1]-2 : 2*pairs[i][1]]))

In [ ]:
tf.keras.backend.set_floatx('float64')
delta_t = 1/60

In [ ]:
# average model
def PP():
  
  inp = keras.layers.Input(shape = (2, ) , dtype = tf.float64)
  
  
  out = keras.layers.Dense(13, activation = 'sigmoid')(inp)

  
  out = keras.layers.Dense(12, activation = 'sigmoid')(out)

  
  out = keras.layers.Dense(10, activation = 'sigmoid')(out)

  
  out = keras.layers.Dense(10,)(out)

  out = keras.layers.Dense(1)(out)
  model = keras.Model([inp], [out])
  return model

In [ ]:
mod = PP()
mod.summary()

In [ ]:
def loss_fn(output, next_output):
  loss_mag = tf.reduce_mean(tf.losses.mean_squared_error(output, next_output))
  #loss_ang = tf.reduce_mean(tf.losses.mean_squared_error(output[:,1:2], next_output[:,1:2]))
  loss = loss_mag
  #print(":: total loss : ", loss)
 
  return loss

In [ ]:
lst_IVs = [BV12, BV13, BV14, BV23, BV24, BV34]

In [ ]:
 
def window_generator(start , finish, pair):
    #abs_pos
    inp_pos_first = pos_arr[start:finish, 2*pair[0]-2:2*pair[0]]
    inp_pos_second = pos_arr[start:finish, 2*pair[1]-2:2*pair[1]]
    rel_pos = np.subtract(inp_pos_first  ,inp_pos_second)
    abs_rel_dist = np.sqrt(np.sum((np.subtract(inp_pos_first  ,inp_pos_second))**2, axis = 1, keepdims = True))
    #abs vel
    inp_vel_first = vel_arr[start:finish, 2*pair[0]-2:2*pair[0]]
    inp_vel_second = vel_arr[start:finish, 2*pair[1]-2:2*pair[1]]
    rel_vel = np.subtract(inp_vel_first  ,inp_vel_second)
    abs_rel_vel = np.sqrt(np.sum((np.subtract(inp_vel_first  ,inp_vel_second))**2, axis = 1, keepdims = True))
    
    len = rel_vel.shape[0]
    #dv
    dv = np.zeros((rel_vel.shape))
    for i in range(len-1):
      dv[i] = rel_vel[i+1]-rel_vel[0]
    
    #app_vel_mag
    app_vel_mag = np.zeros((len, 1))
    for i in range(len):
      if(abs_rel_dist[i,0] > 0):
        app_vel_mag[i, 0] = np.dot(rel_vel[i], rel_pos[i])/abs_rel_dist[i,0]
      else :
        app_vel_mag[i,0] = app_vel_mag[i-1,0]
        

    #abs_dv
    abs_dv = np.sqrt(np.sum((dv)**2, axis = 1, keepdims = True))
 
    #theta_v_x 
    theta_v_x = np.zeros((len, 1))
    for i in range(theta_v_x.shape[0]):  
      v0 = rel_pos[i]
      v1 = rel_vel[i]
      angle = np.arctan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
      theta_v_x[i,0] = np.degrees(angle)
    
    
    #theta_dv_x
    theta_dv_x = np.zeros((len, 1))
    for i in range(theta_dv_x.shape[0]):  
      v0 = rel_pos[i]
      v1 = dv[i]
      angle = np.arctan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
      theta_v_x[i,0] = np.degrees(angle)
    
 
    inp = np.concatenate((abs_rel_dist[:len-1,:], app_vel_mag[:len-1,:], abs_dv[:len-1,:]), axis = 1)
    
    return inp

In [ ]:
j = 1
ind = [0,1,1,0,0,0] 
#ind = [0,0,0,0,0,0]
k = ind[j]
start = lst[j][k][0]
end = lst[j][k][1]
if(j == 1):
  start = start+1
  end = end-6
data = window_generator(start, end+1, pairs[j])
inp = data[0:data.shape[0]-1, 0]
exp_out = data[:, :]
print(inp)
exp_out

In [ ]:
# average model
ind = [0,1,1,0,0,0] 
#ind = [0,0,0,0,0,0]
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
non_zeros_interaction_pairs = [0,0,0,0,0,0]
for i in range(20000):
  print('iteration ;:::: ', i)
  loss = 0
 
  with tf.GradientTape() as Tape:
    for j in range(1, 6):
      k = ind[j]
      non_zeros_interaction_pairs[j] = 1
      start = lst[j][k][0]
      end = lst[j][k][1]
      if(j == 1):
        start = start+1
        end = end-6
      data = window_generator(start, end+1, pairs[j])
      inp = data[:, 0:2]
      exp_out = data[:, 2:]
      #bv = BV_window_generator(start, end, j)
      
      output = mod([ inp])
      output = tf.multiply(output, lst_IVs[j])
      loss_temp = loss_fn(output, exp_out)
      loss += loss_temp
  print("INFO :: " , loss)
  #print(lst_IVs[0])
  diff_var_lst = [mod.trainable_variables]
  
  for m in range(6):
    if (non_zeros_interaction_pairs[m] == 1):
      diff_var_lst.append(lst_IVs[m])
  grad = Tape.gradient(loss, diff_var_lst)
  #print(lst_IVs)
  #optimizer.apply_gradients(zip(grad[0], mod.trainable_variables))
  optimizer.apply_gradients(zip(grad[1:], diff_var_lst[1:]))
  
  
  
  IVs = np.concatenate((np.asarray([non_zeros_interaction_pairs]), keras.backend.get_value(BV12), keras.backend.get_value(BV13), keras.backend.get_value(BV14), keras.backend.get_value(BV23), keras.backend.get_value(BV24), keras.backend.get_value(BV34)), axis = 1)
  mod.save('/content/drive/My Drive/VIPUL COLAB AND DATA/' + 'm' + marker + '.hdf5')
  np.save('/content/drive/My Drive/VIPUL COLAB AND DATA/IVs' + marker + '.npy',IVs)